In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# To load the environment variable defined in the .env file
from dotenv import load_dotenv
load_dotenv();

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.calibration import CalibrationDisplay
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc

import src.data.NHLDataManager as DataManager
import src.features.build_features as FeaturesManager

In [5]:
seasons_year = [2015, 2016, 2017, 2018]
season_type = "Regular"
features_data_df = FeaturesManager.build_features(seasons_year, season_type, with_player_stats=True, with_strength_stats=True)
# features_data_df.drop(columns=['Team', 'Shooter', 'Goalie', 'Shooter ID', 'Goalie ID', 'X', 'Y', 'Last event X', 'Last event Y'], inplace=True)
features_data_df.head(10)

/home/fidaa/anaconda3/envs/NHL/lib/python3.9/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/home/fidaa/anaconda3/envs/NHL/lib/python3.9/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/home/fidaa/anaconda3/envs/NHL/lib/python3.9/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/home/fidaa/anaconda3/envs/NHL/lib/python3.9/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
/home/fidaa/anaconda3/envs/NHL/lib/python3.9/site-packages/numpy/linalg/linalg.py:2154: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


,Game ID,Event Index,Period,Team,Type,Shot Type,Shooter,Shooter ID,Goalie,Goalie ID,...,Game seconds,Last event angle,Rebound,Change in Shot Angle,Speed From Previous Event,Shooter Goal Ratio Last Season,Goalie Goal Ratio Last Season,Num players With,Num players Against,Elapsed time since Power Play
0,2015020001,7,1,Toronto Maple Leafs (TOR),Shot,Wrist Shot,Brad Boyes,8468504.0,Carey Price,8471679.0,...,51,10.525108,0.0,0.000000,14.025066,0.092715,0.066564,5.0,5.0,0.0
1,2015020001,9,1,Toronto Maple Leafs (TOR),Shot,Snap Shot,Nazem Kadri,8475172.0,Carey Price,8471679.0,...,65,28.300756,0.0,0.000000,7.211103,0.102273,0.066564,5.0,5.0,0.0
2,2015020001,10,1,Toronto Maple Leafs (TOR),Shot,Snap Shot,Nazem Kadri,8475172.0,Carey Price,8471679.0,...,66,0.000000,1.0,4.085617,4.123106,0.102273,0.066564,5.0,5.0,0.0
3,2015020001,13,1,Toronto Maple Leafs (TOR),Shot,Wrist Shot,Daniel Winnik,8471476.0,Carey Price,8471679.0,...,113,-93.270488,0.0,0.000000,4.422515,0.092784,0.066564,5.0,5.0,0.0
4,2015020001,21,1,Toronto Maple Leafs (TOR),Shot,Tip-In,Joffrey Lupul,8470207.0,Carey Price,8471679.0,...,157,47.726311,0.0,0.000000,4.494441,0.103093,0.066564,5.0,5.0,0.0
5,2015020001,24,1,Montréal Canadiens (MTL),Goal,Wrist Shot,Max Pacioretty,8474157.0,Jonathan Bernier,8473541.0,...,189,-18.434949,0.0,0.000000,3.824492,0.122517,0.087608,5.0,5.0,0.0
6,2015020001,29,1,Montréal Canadiens (MTL),Shot,Wrist Shot,Jeff Petry,8473507.0,Jonathan Bernier,8473541.0,...,234,14.036243,0.0,0.000000,9.055385,0.055556,0.087608,5.0,5.0,0.0
7,2015020001,45,1,Montréal Canadiens (MTL),Shot,Backhand,Tomas Plekanec,8469521.0,Jonathan Bernier,8473541.0,...,374,-7.926927,0.0,0.000000,3.211928,0.104839,0.087608,4.0,4.0,0.0
8,2015020001,46,1,Montréal Canadiens (MTL),Shot,Wrist Shot,Tom Gilbert,8470104.0,Jonathan Bernier,8473541.0,...,410,-45.763898,1.0,19.198847,0.626695,0.057143,0.087608,4.0,4.0,0.0
9,2015020001,54,1,Toronto Maple Leafs (TOR),Shot,Wrist Shot,Shawn Matthias,8473574.0,Carey Price,8471679.0,...,493,-7.926927,0.0,0.000000,24.543046,0.136364,0.066564,5.0,5.0,0.0


In [9]:
# Q1
features_data_df[['Period', 'Period seconds', 'Game seconds', 'st_X', 'st_Y', 'Shot distance', 'Shot angle', 'Shot Type']].sample(10)


,Period,Period seconds,Game seconds,st_X,st_Y,Shot distance,Shot angle,Shot Type
136831,3,944.0,3344,54.0,-1.0,35.014283,-1.636577,Snap Shot
48442,2,1023.0,2223,53.0,3.0,36.124784,4.763642,Wrist Shot
147804,2,657.0,1857,55.0,13.0,36.400549,20.924502,Wrist Shot
65899,2,929.0,2129,72.0,-19.0,25.495098,-48.179830,Wrist Shot
13041,3,126.0,2526,-73.0,-12.0,162.443836,-4.236395,Snap Shot
213012,1,847.0,847,77.0,-10.0,15.620499,-39.805571,Backhand
293131,3,359.0,2759,69.0,1.0,20.024984,2.862405,Tip-In
368294,1,1199.0,1199,81.0,-3.0,8.544004,-20.556045,Tip-In
25563,1,593.0,593,73.0,-11.0,19.416488,-34.508523,Snap Shot
314844,2,1151.0,2351,54.0,18.0,39.357337,27.216112,Wrist Shot


In [13]:
# Q2 
features_data_df[['Last event type', 'Last event elapsed time', 'Last event distance', 'Last event st_X', 'Last event st_Y']].sample(10)

,Last event type,Last event elapsed time,Last event distance,Last event st_X,Last event st_Y
317064,Blocked Shot,1.0,5.000000,81.0,2.0
280182,Hit,3.0,46.238512,69.0,37.0
129687,Giveaway,2.0,30.479501,35.0,25.0
85674,Missed Shot,2.0,24.839485,67.0,9.0
4181,Faceoff,6.0,21.470911,-69.0,-22.0
205624,Faceoff,10.0,89.442719,0.0,0.0
134292,Hit,12.0,105.380264,-13.0,-40.0
320335,Faceoff,8.0,88.022724,-69.0,22.0
114852,Hit,1.0,42.011903,75.0,40.0
279880,Blocked Shot,15.0,141.226768,-68.0,6.0


In [15]:
# Q3
features_data_df[['Rebound', 'Change in Shot Angle', 'Speed From Previous Event']].sample(10)

,Rebound,Change in Shot Angle,Speed From Previous Event
324254,0.0,0.000000,6.108692
226551,0.0,0.000000,6.236591
217751,1.0,41.054814,6.708204
28638,0.0,0.000000,1.092591
93864,1.0,29.261199,1.226020
128524,1.0,9.509610,0.534000
227023,0.0,0.000000,6.450189
157232,0.0,0.000000,2.499653
168880,0.0,0.000000,6.885220
245473,1.0,8.763628,0.971825


In [16]:
# Q4
features_data_df[[ 'Num players With', 'Num players Against', 'Elapsed time since Power Play']].sample(10)
features_data_df.columns

Index(['Game ID', 'Event Index', 'Period', 'Team', 'Type', 'Shot Type',
       'Shooter', 'Shooter ID', 'Goalie', 'Goalie ID', 'Strength', 'X', 'Y',
       'Last event type', 'Last event X', 'Last event Y',
       'Last event elapsed time', 'Last event distance', 'st_X', 'st_Y',
       'Last event st_X', 'Last event st_Y', 'Shot distance', 'Shot angle',
       'Is Goal', 'Is Empty', 'Period seconds', 'Game seconds',
       'Last event angle', 'Rebound', 'Change in Shot Angle',
       'Speed From Previous Event', 'Shooter Goal Ratio Last Season',
       'Goalie Goal Ratio Last Season', 'Num players With',
       'Num players Against', 'Elapsed time since Power Play'],
      dtype='object')